In [0]:
import psycopg2

# Connection details
db_host = "databricks-postgres-db.cd424ookkd9v.ap-south-1.rds.amazonaws.com"
db_port = "5432"
db_name = "postgres"
db_user = "postgres"
db_password = "Abcabc123"

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    dbname=db_name,
    user=db_user,
    password=db_password
)
conn.autocommit = True
cursor = conn.cursor()

# SQL: Drop tables if they exist
drop_commands = [
    "DROP TABLE IF EXISTS merchant_transaction_counts CASCADE;",
    "DROP TABLE IF EXISTS customer_merchant_stats CASCADE;",
    "DROP TABLE IF EXISTS merchant_gender_stats CASCADE;"
]

# SQL: Create tables with updated gender column definition
create_commands = [
    """
    CREATE TABLE merchant_transaction_counts (
        merchant_id VARCHAR(255) PRIMARY KEY,
        transaction_count BIGINT DEFAULT 0
    );
    """,
    """
    CREATE TABLE customer_merchant_stats (
        customer_id VARCHAR(255),
        merchant_id VARCHAR(255),
        transaction_count BIGINT DEFAULT 0,
        weight_sum DECIMAL(15,2) DEFAULT 0,
        weight_count BIGINT DEFAULT 0,
        amount_sum DECIMAL(15,2) DEFAULT 0,
        amount_count BIGINT DEFAULT 0,
        PRIMARY KEY (customer_id, merchant_id)
    );
    """,
    """
    CREATE TABLE merchant_gender_stats (
        merchant_id VARCHAR(255),
        gender VARCHAR(10) CHECK (
            gender IN ('M', 'F', 'Male', 'Female', 'MALE', 'FEMALE', 'Other', 'Unknown', '')
        ),
        customer_count BIGINT DEFAULT 0,
        PRIMARY KEY (merchant_id, gender)
    );
    """
]

# SQL: Create indexes
index_commands = [
    "CREATE INDEX idx_customer_merchant_stats_merchant ON customer_merchant_stats(merchant_id);",
    "CREATE INDEX idx_merchant_gender_stats_merchant ON merchant_gender_stats(merchant_id);",
    "CREATE INDEX idx_customer_merchant_stats_customer ON customer_merchant_stats(customer_id);"
]

# Drop old tables
print("Dropping existing tables...")
for cmd in drop_commands:
    try:
        cursor.execute(cmd)
        print(f"Dropped: {cmd.strip()}")
    except Exception as e:
        print(f"Error dropping table: {e}")

# Create new tables
print("\nCreating new tables...")
for i, cmd in enumerate(create_commands, start=1):
    try:
        cursor.execute(cmd)
        print(f"Table {i} created successfully.")
    except Exception as e:
        print(f"Error creating table {i}: {e}")

# Create indexes
print("\nCreating indexes...")
for cmd in index_commands:
    try:
        cursor.execute(cmd)
        print(f"Index created: {cmd.strip()}")
    except Exception as e:
        print(f"Error creating index: {e}")

# Verify tables exist
print("\nVerifying created tables...")
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    AND table_name IN ('merchant_transaction_counts', 'customer_merchant_stats', 'merchant_gender_stats')
    ORDER BY table_name;
""")
tables = cursor.fetchall()
print("Tables present in database:")
for table in tables:
    print(f"  - {table[0]}")

# Clean up
cursor.close()
conn.close()
print("\nDatabase setup completed successfully!")


Dropping existing tables...
Dropped: DROP TABLE IF EXISTS merchant_transaction_counts CASCADE;
Dropped: DROP TABLE IF EXISTS customer_merchant_stats CASCADE;
Dropped: DROP TABLE IF EXISTS merchant_gender_stats CASCADE;

Creating new tables...
Table 1 created successfully.
Table 2 created successfully.
Table 3 created successfully.

Creating indexes...
Index created: CREATE INDEX idx_customer_merchant_stats_merchant ON customer_merchant_stats(merchant_id);
Index created: CREATE INDEX idx_merchant_gender_stats_merchant ON merchant_gender_stats(merchant_id);
Index created: CREATE INDEX idx_customer_merchant_stats_customer ON customer_merchant_stats(customer_id);

Verifying created tables...
Tables present in database:
  - customer_merchant_stats
  - merchant_gender_stats
  - merchant_transaction_counts

Database setup completed successfully!
